In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
#Creating Spark Session (It will executed in the Local Not in the Master Node)
spark=SparkSession.builder.master("local[*]").appName("demo").getOrCreate()
spark

23/03/23 06:45:19 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/03/23 06:45:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 06:45:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/23 06:45:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
empDf = spark.read.option("header",True).option("inferSchema",True).csv("../Spark-main/employees.csv")
empDf.show(5)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

In [6]:
deptDf = spark.read.option("header",True).option("inferSchema",True).csv("../Spark-main/departments.csv")
deptDf.show(5)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
|           30|     Purchasing|       114|       1700|
|           40|Human Resources|       203|       2400|
|           50|       Shipping|       121|       1500|
+-------------+---------------+----------+-----------+
only showing top 5 rows



In [8]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |      

In [10]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select("EMPLOYEE_ID", "DEPARTMENT_ID", "DEPARTMENT_NAME").show()

AnalysisException: Reference 'DEPARTMENT_ID' is ambiguous, could be: DEPARTMENT_ID, DEPARTMENT_ID.

In [12]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show()

+-----------+-------------+---------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|
+-----------+-------------+---------------+
|        200|           10| Administration|
|        202|           20|      Marketing|
|        201|           20|      Marketing|
|        119|           30|     Purchasing|
|        118|           30|     Purchasing|
|        117|           30|     Purchasing|
|        116|           30|     Purchasing|
|        115|           30|     Purchasing|
|        114|           30|     Purchasing|
|        203|           40|Human Resources|
|        140|           50|       Shipping|
|        139|           50|       Shipping|
|        138|           50|       Shipping|
|        137|           50|       Shipping|
|        136|           50|       Shipping|
|        135|           50|       Shipping|
|        134|           50|       Shipping|
|        133|           50|       Shipping|
|        132|           50|       Shipping|
|        131|           50|     

In [14]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "left").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show(100)

+-----------+-------------+----------------+
|EMPLOYEE_ID|DEPARTMENT_ID| DEPARTMENT_NAME|
+-----------+-------------+----------------+
|        198|           50|        Shipping|
|        199|           50|        Shipping|
|        200|           10|  Administration|
|        201|           20|       Marketing|
|        202|           20|       Marketing|
|        203|           40| Human Resources|
|        204|           70|Public Relations|
|        205|          110|      Accounting|
|        206|          110|      Accounting|
|        100|           90|       Executive|
|        101|           90|       Executive|
|        102|           90|       Executive|
|        103|           60|              IT|
|        104|           60|              IT|
|        105|           60|              IT|
|        106|           60|              IT|
|        107|           60|              IT|
|        108|          100|         Finance|
|        109|          100|         Finance|
|        1

In [15]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "right").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show(100)

+-----------+-------------+--------------------+
|EMPLOYEE_ID|DEPARTMENT_ID|     DEPARTMENT_NAME|
+-----------+-------------+--------------------+
|        200|           10|      Administration|
|        202|           20|           Marketing|
|        201|           20|           Marketing|
|        119|           30|          Purchasing|
|        118|           30|          Purchasing|
|        117|           30|          Purchasing|
|        116|           30|          Purchasing|
|        115|           30|          Purchasing|
|        114|           30|          Purchasing|
|        203|           40|     Human Resources|
|        140|           50|            Shipping|
|        139|           50|            Shipping|
|        138|           50|            Shipping|
|        137|           50|            Shipping|
|        136|           50|            Shipping|
|        135|           50|            Shipping|
|        134|           50|            Shipping|
|        133|       

In [16]:
empDf.join(deptDf, empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "fullouter").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show(100)

+-----------+-------------+--------------------+
|EMPLOYEE_ID|DEPARTMENT_ID|     DEPARTMENT_NAME|
+-----------+-------------+--------------------+
|        200|           10|      Administration|
|        201|           20|           Marketing|
|        202|           20|           Marketing|
|        114|           30|          Purchasing|
|        115|           30|          Purchasing|
|        116|           30|          Purchasing|
|        117|           30|          Purchasing|
|        118|           30|          Purchasing|
|        119|           30|          Purchasing|
|        203|           40|     Human Resources|
|        198|           50|            Shipping|
|        199|           50|            Shipping|
|        120|           50|            Shipping|
|        121|           50|            Shipping|
|        122|           50|            Shipping|
|        123|           50|            Shipping|
|        124|           50|            Shipping|
|        125|       

### Join With Restrictions

In [19]:
empDf.show(5)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

In [20]:
deptDf.show(5)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
|           30|     Purchasing|       114|       1700|
|           40|Human Resources|       203|       2400|
|           50|       Shipping|       121|       1500|
+-------------+---------------+----------+-----------+
only showing top 5 rows



In [17]:
deptDf.filter(deptDf.MANAGER_ID != "-").show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|        - |       1700|
|          130|       Corporate Tax|        - |       1700|
|          140|  Control And Credit|    

In [22]:
from pyspark.sql.functions import col
empDf.alias("emp1").join(empDf.alias("emp2") , col("emp1.manager_id") == col("emp2.employee_id"), "inner").select(col("emp1.manager_id"), col("emp2.first_name"), col("emp2.last_name")).show(100)

+----------+----------+---------+
|manager_id|first_name|last_name|
+----------+----------+---------+
|       201|   Michael|Hartstein|
|       205|   Shelley|  Higgins|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       101|     Neena|  Kochhar|
|       101|     Neena|  Kochhar|
|       101|     Neena|  Kochhar|
|       101|     Neena|  Kochhar|
|       101|     Neena|  Kochhar|
|       102|       Lex|  De Haan|
|       103| Alexander|   Hunold|
|       103| Alexander|   Hunold|
|       103| Alexander|   Hunold|
|       103| Alexander|   Hunold|
|       108|     Nancy|Greenberg|
|       108|     Nancy|Greenberg|
|       108|     Nancy|Greenberg|
|       108|     Nancy|Greenberg|
|       108|     Nancy|Greenberg|
|       114|  

In [23]:
empDf.alias("emp1").join(empDf.alias("emp2") , col("emp1.manager_id") == col("emp2.employee_id"), "inner").select(col("emp1.manager_id"), col("emp2.first_name"), col("emp2.last_name")).dropDuplicates().show(100)

+----------+----------+---------+
|manager_id|first_name|last_name|
+----------+----------+---------+
|       122|     Payam| Kaufling|
|       101|     Neena|  Kochhar|
|       100|    Steven|     King|
|       205|   Shelley|  Higgins|
|       114|       Den| Raphaely|
|       103| Alexander|   Hunold|
|       124|     Kevin|  Mourgos|
|       120|   Matthew|    Weiss|
|       123|    Shanta|  Vollman|
|       121|      Adam|    Fripp|
|       108|     Nancy|Greenberg|
|       102|       Lex|  De Haan|
|       201|   Michael|Hartstein|
+----------+----------+---------+



In [26]:
empDf.join(deptDf, (empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID) & (deptDf.LOCATION_ID == 1700), "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show()

+-----------+-------------+---------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|
+-----------+-------------+---------------+
|        200|           10| Administration|
|        119|           30|     Purchasing|
|        118|           30|     Purchasing|
|        117|           30|     Purchasing|
|        116|           30|     Purchasing|
|        115|           30|     Purchasing|
|        114|           30|     Purchasing|
|        102|           90|      Executive|
|        101|           90|      Executive|
|        100|           90|      Executive|
|        113|          100|        Finance|
|        112|          100|        Finance|
|        111|          100|        Finance|
|        110|          100|        Finance|
|        109|          100|        Finance|
|        108|          100|        Finance|
|        206|          110|     Accounting|
|        205|          110|     Accounting|
+-----------+-------------+---------------+



In [27]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
location_data = [(1700, "INDIA"), (1800, "USA")]
schema = StructType([ StructField("LOCATION_ID",IntegerType(),True), StructField("LOCATION_NAME",StringType(),True) ])
locDf = spark.createDataFrame(data=location_data,schema=schema)
locDf.printSchema()

root
 |-- LOCATION_ID: integer (nullable = true)
 |-- LOCATION_NAME: string (nullable = true)



In [28]:
locDf.show()

+-----------+-------------+
|LOCATION_ID|LOCATION_NAME|
+-----------+-------------+
|       1700|        INDIA|
|       1800|          USA|
+-----------+-------------+



### Multiple Join

In [31]:
empDf.join(deptDf, (empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID) & (deptDf.LOCATION_ID == 1700), "inner").join(locDf, deptDf.LOCATION_ID == locDf.LOCATION_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME, locDf.LOCATION_NAME).show()

+-----------+-------------+---------------+-------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|LOCATION_NAME|
+-----------+-------------+---------------+-------------+
|        205|          110|     Accounting|        INDIA|
|        206|          110|     Accounting|        INDIA|
|        108|          100|        Finance|        INDIA|
|        109|          100|        Finance|        INDIA|
|        110|          100|        Finance|        INDIA|
|        111|          100|        Finance|        INDIA|
|        112|          100|        Finance|        INDIA|
|        113|          100|        Finance|        INDIA|
|        100|           90|      Executive|        INDIA|
|        101|           90|      Executive|        INDIA|
|        102|           90|      Executive|        INDIA|
|        114|           30|     Purchasing|        INDIA|
|        115|           30|     Purchasing|        INDIA|
|        116|           30|     Purchasing|        INDIA|
|        117| 

### UDF

In [36]:
from pyspark.sql.functions import udf

def upperCase(in_str):
    out_str = in_str.upper()
    return out_str

print(upperCase("hello"))
upperCaseUDF = udf(lambda z : upperCase(z) , StringType())
empDf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseUDF(col("FIRST_NAME")), upperCaseUDF(col("LAST_NAME"))).show()

HELLO
+-----------+----------+---------+--------------------+-------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|<lambda>(FIRST_NAME)|<lambda>(LAST_NAME)|
+-----------+----------+---------+--------------------+-------------------+
|        198|    Donald| OConnell|              DONALD|           OCONNELL|
|        199|   Douglas|    Grant|             DOUGLAS|              GRANT|
|        200|  Jennifer|   Whalen|            JENNIFER|             WHALEN|
|        201|   Michael|Hartstein|             MICHAEL|          HARTSTEIN|
|        202|       Pat|      Fay|                 PAT|                FAY|
|        203|     Susan|   Mavris|               SUSAN|             MAVRIS|
|        204|   Hermann|     Baer|             HERMANN|               BAER|
|        205|   Shelley|  Higgins|             SHELLEY|            HIGGINS|
|        206|   William|    Gietz|             WILLIAM|              GIETZ|
|        100|    Steven|     King|              STEVEN|               KING|
|     

In [37]:
@udf(returnType=StringType())
def upperCaseNew(in_str):
    out_str = in_str.upper()
    return out_str

In [38]:
empDf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseNew(col("FIRST_NAME")), upperCaseNew(col("LAST_NAME"))).show()

+-----------+----------+---------+------------------------+-----------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|upperCaseNew(FIRST_NAME)|upperCaseNew(LAST_NAME)|
+-----------+----------+---------+------------------------+-----------------------+
|        198|    Donald| OConnell|                  DONALD|               OCONNELL|
|        199|   Douglas|    Grant|                 DOUGLAS|                  GRANT|
|        200|  Jennifer|   Whalen|                JENNIFER|                 WHALEN|
|        201|   Michael|Hartstein|                 MICHAEL|              HARTSTEIN|
|        202|       Pat|      Fay|                     PAT|                    FAY|
|        203|     Susan|   Mavris|                   SUSAN|                 MAVRIS|
|        204|   Hermann|     Baer|                 HERMANN|                   BAER|
|        205|   Shelley|  Higgins|                 SHELLEY|                HIGGINS|
|        206|   William|    Gietz|                 WILLIAM|                 

### Windows Functions

In [39]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy("SALARY")
empDf.withColumn("salary_rank", rank().over(windowSpec)).select("DEPARTMENT_ID","SALARY","salary_rank").show(100)

+-------------+------+-----------+
|DEPARTMENT_ID|SALARY|salary_rank|
+-------------+------+-----------+
|           10|  4400|          1|
|           20|  6000|          1|
|           20| 13000|          2|
|           30|  2500|          1|
|           30|  2600|          2|
|           30|  2800|          3|
|           30|  2900|          4|
|           30|  3100|          5|
|           30| 11000|          6|
|           40|  6500|          1|
|           50|  2100|          1|
|           50|  2200|          2|
|           50|  2200|          2|
|           50|  2400|          4|
|           50|  2400|          4|
|           50|  2500|          6|
|           50|  2500|          6|
|           50|  2600|          8|
|           50|  2600|          8|
|           50|  2700|         10|
|           50|  2700|         10|
|           50|  2800|         12|
|           50|  2900|         13|
|           50|  3200|         14|
|           50|  3200|         14|
|           50|  330

In [40]:
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy(col("SALARY").desc())
empDf.withColumn("salary_rank", rank().over(windowSpec)).select("DEPARTMENT_ID","SALARY","salary_rank").show(100)

+-------------+------+-----------+
|DEPARTMENT_ID|SALARY|salary_rank|
+-------------+------+-----------+
|           10|  4400|          1|
|           20| 13000|          1|
|           20|  6000|          2|
|           30| 11000|          1|
|           30|  3100|          2|
|           30|  2900|          3|
|           30|  2800|          4|
|           30|  2600|          5|
|           30|  2500|          6|
|           40|  6500|          1|
|           50|  8200|          1|
|           50|  8000|          2|
|           50|  7900|          3|
|           50|  6500|          4|
|           50|  5800|          5|
|           50|  3600|          6|
|           50|  3300|          7|
|           50|  3300|          7|
|           50|  3200|          9|
|           50|  3200|          9|
|           50|  2900|         11|
|           50|  2800|         12|
|           50|  2700|         13|
|           50|  2700|         13|
|           50|  2600|         15|
|           50|  260

In [41]:
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy(col("SALARY").desc())
empDf.withColumn("SUM", sum("SALARY").over(windowSpec)).select("DEPARTMENT_ID","SALARY","SUM").show(100)

+-------------+------+-----+
|DEPARTMENT_ID|SALARY|  SUM|
+-------------+------+-----+
|           10|  4400| 4400|
|           20| 13000|13000|
|           20|  6000|19000|
|           30| 11000|11000|
|           30|  3100|14100|
|           30|  2900|17000|
|           30|  2800|19800|
|           30|  2600|22400|
|           30|  2500|24900|
|           40|  6500| 6500|
|           50|  8200| 8200|
|           50|  8000|16200|
|           50|  7900|24100|
|           50|  6500|30600|
|           50|  5800|36400|
|           50|  3600|40000|
|           50|  3300|46600|
|           50|  3300|46600|
|           50|  3200|53000|
|           50|  3200|53000|
|           50|  2900|55900|
|           50|  2800|58700|
|           50|  2700|64100|
|           50|  2700|64100|
|           50|  2600|69300|
|           50|  2600|69300|
|           50|  2500|74300|
|           50|  2500|74300|
|           50|  2400|79100|
|           50|  2400|79100|
|           50|  2200|83500|
|           50

In [42]:
windowSpec = Window.partitionBy("DEPARTMENT_ID")
empDf.withColumn("SUM", sum("SALARY").over(windowSpec)).select("DEPARTMENT_ID","SALARY","SUM").show(100)

+-------------+------+-----+
|DEPARTMENT_ID|SALARY|  SUM|
+-------------+------+-----+
|           10|  4400| 4400|
|           20| 13000|19000|
|           20|  6000|19000|
|           30| 11000|24900|
|           30|  3100|24900|
|           30|  2900|24900|
|           30|  2800|24900|
|           30|  2600|24900|
|           30|  2500|24900|
|           40|  6500| 6500|
|           50|  2600|85600|
|           50|  2600|85600|
|           50|  8000|85600|
|           50|  8200|85600|
|           50|  7900|85600|
|           50|  6500|85600|
|           50|  5800|85600|
|           50|  3200|85600|
|           50|  2700|85600|
|           50|  2400|85600|
|           50|  2200|85600|
|           50|  3300|85600|
|           50|  2800|85600|
|           50|  2500|85600|
|           50|  2100|85600|
|           50|  3300|85600|
|           50|  2900|85600|
|           50|  2400|85600|
|           50|  2200|85600|
|           50|  3600|85600|
|           50|  3200|85600|
|           50